In [ ]:
from tensorflow.keras.layers import GRU

# Define the GRU model inputs
input_seq = Input(shape=(max_len, len(amino_acid_mapping)))

# Define GRU layers
x = GRU(256, return_sequences=True)(input_seq)
x = GRU(256)(x)

# Define the two outputs
broad_output = Dense(len(broad_label_encoder.classes_), activation='softmax', name='broad_output')(x)
detailed_output = Dense(len(protein_label_encoder.classes_), activation='softmax', name='detailed_output')(x)

# Create the GRU model
gru_model = Model(inputs=input_seq, outputs=[broad_output, detailed_output])

# Compile the GRU model
gru_model.compile(optimizer='adam',
              loss={'broad_output': 'categorical_crossentropy', 'detailed_output': 'categorical_crossentropy'},
              metrics={'broad_output': 'accuracy', 'detailed_output': 'accuracy'})

# Train the GRU model
history_gru = gru_model.fit(
    X_train,
    {'broad_output': y_broad_train, 'detailed_output': y_train},
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the GRU model
gru_scores = gru_model.evaluate(X_test, {'broad_output': y_broad_test, 'detailed_output': y_test}, verbose=1)

# Print classification report for GRU
print(f"GRU - Broad classification accuracy: {gru_scores[3]:.4f}")
print(f"GRU - Detailed classification accuracy: {gru_scores[4]:.4f}")
